<a href="https://colab.research.google.com/github/fun-debug/demo/blob/main/d2l/chapter_computer-vision/transposed_conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Feb 11 13:35:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install d2l matplotlib-inline

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.6/112.6 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 100.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 KB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 965.4/965.4 KB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.6/83.6 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 84.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.6
    Uninstalling numpy-1.21.6:
      Successfully uninstalled numpy-1.21.6
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.

In [3]:
!pip install matplotlib==3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.3/36.3 MB 34.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for matplotlib: filename=matplotlib-3.0.0-cp38-cp38-linux_x86_64.whl size=11844445 sha256=118ffcdf8133d7d8945cc9f82841c1292a60af039cf38eb85470567cef9764a7
  Stored in directory: /root/.cache/pip/wheels/a9/1e/ed/3e09c67472517aa5d81f8c15de6c81162d249ebe442e4f3007
Successfully built matplotlib
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.1
    Uninstalling matplotlib-3.5.1:
      Successfully uninstalled matplotlib-3.5.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires matplotlib!=3.0.0,>=2.0.2, but you have matplotlib 3.0.0 which is incompatible.
scikit-image 0.18.3 req

In [4]:
import torch
from torch import nn
from d2l import torch as d2l

In [5]:
def trans_conv(X,K):
  h,w=K.shape
  Y=torch.zeros((X.shape[0]+h-1,X.shape[1]+w-1))
  for i in range(X.shape[0]):
    for j in range(X.shape[1]):
      Y[i:i+h,j:j+w] +=X[i][j]*K
  return Y

In [6]:
X=torch.tensor([[0.0,1.0],[2.0,3.0]])
K=torch.tensor([[0.0,1.0],[2.0,3.0]])
trans_conv(X,K)

tensor([[ 0.,  0.,  1.],
        [ 0.,  4.,  6.],
        [ 4., 12.,  9.]])

In [8]:
X,K=X.reshape(1,1,2,2),K.reshape(1,1,2,2)
tconv=nn.ConvTranspose2d(1,1,kernel_size=2,bias=False)
tconv.weight.data=K
tconv(X)

tensor([[[[ 0.,  0.,  1.],
          [ 0.,  4.,  6.],
          [ 4., 12.,  9.]]]], grad_fn=<ConvolutionBackward0>)

In [9]:
tconv=nn.ConvTranspose2d(1,1,kernel_size=2,padding=1,bias=False)
tconv.weight.data=K
tconv(X)

tensor([[[[4.]]]], grad_fn=<ConvolutionBackward0>)

In [11]:
tconv=nn.ConvTranspose2d(1,1,kernel_size=2,stride=2,bias=False)
tconv.weight.data=K
tconv(X)

tensor([[[[0., 0., 0., 1.],
          [0., 0., 2., 3.],
          [0., 2., 0., 3.],
          [4., 6., 6., 9.]]]], grad_fn=<ConvolutionBackward0>)

In [13]:
X=torch.rand(size=(1,10,16,16))
conv=nn.Conv2d(10,20,kernel_size=5,padding=2,stride=3)
tconv=nn.ConvTranspose2d(20,10,kernel_size=5,padding=2,stride=3)
tconv(conv(X)).shape == X.shape

True

In [15]:
X=torch.arange(9.0).reshape(3,3)
K=torch.tensor([[1.0,2.0],[3.0,4.0]])
Y=d2l.corr2d(X,K)
Y

tensor([[27., 37.],
        [57., 67.]])

In [16]:
def kernel2matrix(K):
  k,W=torch.zeros(5),torch.zeros((4,9))
  k[0:2],k[3:5]=K[0,:],K[1,:]
  W[0,:5],W[1,1:6],W[2,3:8],W[3,4:]=k,k,k,k
  return W
W=kernel2matrix(K)
W

tensor([[1., 2., 0., 3., 4., 0., 0., 0., 0.],
        [0., 1., 2., 0., 3., 4., 0., 0., 0.],
        [0., 0., 0., 1., 2., 0., 3., 4., 0.],
        [0., 0., 0., 0., 1., 2., 0., 3., 4.]])

In [18]:
Y == torch.matmul(W,X.reshape(-1)).reshape(2,2)

tensor([[True, True],
        [True, True]])

In [20]:
Z = trans_conv(Y,K)
Z == torch.matmul(W.T,Y.reshape(-1)).reshape(3,3)

tensor([[True, True, True],
        [True, True, True],
        [True, True, True]])